# Gridsearch Cluster all small dag generation evaluation

In [1]:
%load_ext autoreload
%autoreload 2
from clustercausal.experiments.ExperimentRunner import ExperimentRunner
import itertools
import yaml
import os
import pickle
import pandas as pd
from clustercausal.utils.Utils import *
from clustercausal.experiments.Utils import load_experiment_folder
pd.set_option('display.max_columns', None)

Detecting 1 CUDA device(s).
2023-11-08 22:02:43,693 - c:\Users\JanMarco\MLProjects\Masterthesis\masterthesis\mthesis\lib\site-packages\rpy2\situation.py[line:41] - INFO: cffi mode is CFFI_MODE.ANY
2023-11-08 22:02:43,693 - c:\Users\JanMarco\MLProjects\Masterthesis\masterthesis\mthesis\lib\site-packages\rpy2\situation.py[line:218] - INFO: R home found: C:\Program Files\R\R-4.3.1
2023-11-08 22:02:43,743 - c:\Users\JanMarco\MLProjects\Masterthesis\masterthesis\mthesis\lib\site-packages\rpy2\rinterface_lib\embedded.py[line:20] - INFO: Default options to initialize R: rpy2, --quiet, --no-save
IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
# Can take a longer time depending on the numer of experiments, ca. 5sec for 100 experiments
data = load_experiment_folder('clustercausal\experiments\_results\debug_test')

### Data exploration

In [27]:
i = 10
data[i:i+10]

,base_adj_f1_score,base_adj_false_negative,base_adj_false_positive,base_adj_precision,base_adj_recall,base_adj_true_negative,base_adj_true_positive,base_arrow_f1_score,base_arrow_f1_score_ce,base_arrow_false_negative,base_arrow_false_negative_ce,base_arrow_false_positive,base_arrow_false_positive_ce,base_arrow_precision,base_arrow_precision_ce,base_arrow_recall,base_arrow_recall_ce,base_arrow_true_negative,base_arrow_true_negative_ce,base_arrow_true_positive,base_arrow_true_positive_ce,base_base_shd,base_sid_lower,cluster_adj_f1_score,cluster_adj_false_negative,cluster_adj_false_positive,cluster_adj_precision,cluster_adj_recall,cluster_adj_true_negative,cluster_adj_true_positive,cluster_arrow_f1_score,cluster_arrow_f1_score_ce,cluster_arrow_false_negative,cluster_arrow_false_negative_ce,cluster_arrow_false_positive,cluster_arrow_false_positive_ce,cluster_arrow_precision,cluster_arrow_precision_ce,cluster_arrow_recall,cluster_arrow_recall_ce,cluster_arrow_true_negative,cluster_arrow_true_negative_ce,cluster_arrow_true_positive,cluster_arrow_true_positive_ce,cluster_cluster_shd,cluster_sid_lower,pruned_base_adj_f1_score,pruned_base_adj_false_negative,pruned_base_adj_false_positive,pruned_base_adj_precision,pruned_base_adj_recall,pruned_base_adj_true_negative,pruned_base_adj_true_positive,pruned_base_arrow_f1_score,pruned_base_arrow_f1_score_ce,pruned_base_arrow_false_negative,pruned_base_arrow_false_negative_ce,pruned_base_arrow_false_positive,pruned_base_arrow_false_positive_ce,pruned_base_arrow_precision,pruned_base_arrow_precision_ce,pruned_base_arrow_recall,pruned_base_arrow_recall_ce,pruned_base_arrow_true_negative,pruned_base_arrow_true_negative_ce,pruned_base_arrow_true_positive,pruned_base_arrow_true_positive_ce,pruned_base_pruned_base_shd,pruned_base_sid_lower,alpha,cluster_connectivity,dag_method,distribution_type,edge_ratios,empty_graph_shd,indep_test,n_clusters,n_edges,n_nodes,noise_scale,sample_size,scm_method,seed,true_sid_lower,true_sid_upper,weight_range,base_total_true,base_total_false,pruned_base_total_true,pruned_base_total_false,cluster_total_true,cluster_total_false
10,0.888889,2.0,0.0,1.000000,0.800000,35.0,8.0,0.571429,0.666667,6.0,4.0,0.0,0.0,1.000000,1.000000,0.400000,0.500000,90.0,92.0,4.0,4.0,2.0,NaN,0.947368,1.0,0.0,1.000000,0.900000,35.0,9.0,0.823529,0.875000,3.0,2.0,0.0,0.0,1.000000,1.000000,0.700000,0.777778,90.0,91.0,7.0,7.0,1.0,NaN,0.888889,2.0,0.0,1.000000,0.800000,35.0,8.0,0.571429,0.666667,6.0,4.0,0.0,0.0,1.000000,1.000000,0.400000,0.500000,90.0,92.0,4.0,4.0,2.0,NaN,0.01,1.000000,erdos_renyi,gumbel,"[0.45, 0.14, 0.14]",10.0,fisherz,2.0,10.0,10.0,1.0,1000.0,linear,None,NaN,NaN,"(-1, 2)",43.0,2.0,43.0,2.0,44.0,1.0
11,0.888889,2.0,0.0,1.000000,0.800000,35.0,8.0,0.571429,0.666667,6.0,4.0,0.0,0.0,1.000000,1.000000,0.400000,0.500000,90.0,92.0,4.0,4.0,2.0,NaN,0.888889,2.0,0.0,1.000000,0.800000,35.0,8.0,0.823529,0.933333,3.0,1.0,0.0,0.0,1.000000,1.000000,0.700000,0.875000,90.0,92.0,7.0,7.0,2.0,NaN,0.888889,2.0,0.0,1.000000,0.800000,35.0,8.0,0.571429,0.666667,6.0,4.0,0.0,0.0,1.000000,1.000000,0.400000,0.500000,90.0,92.0,4.0,4.0,2.0,NaN,0.05,1.000000,erdos_renyi,gumbel,"[0.07, 0.33, 0.33]",10.0,fisherz,2.0,10.0,10.0,1.0,1000.0,linear,None,NaN,NaN,"(-1, 2)",43.0,2.0,43.0,2.0,43.0,2.0
12,1.000000,0.0,0.0,1.000000,1.000000,38.0,7.0,0.615385,0.615385,3.0,3.0,2.0,2.0,0.666667,0.666667,0.571429,0.571429,91.0,91.0,4.0,4.0,2.0,NaN,1.000000,0.0,0.0,1.000000,1.000000,38.0,7.0,0.923077,0.923077,1.0,1.0,0.0,0.0,1.000000,1.000000,0.857143,0.857143,93.0,93.0,6.0,6.0,0.0,NaN,1.000000,0.0,0.0,1.000000,1.000000,38.0,7.0,0.615385,0.615385,3.0,3.0,2.0,2.0,0.666667,0.666667,0.571429,0.571429,91.0,91.0,4.0,4.0,2.0,NaN,0.10,1.000000,erdos_renyi,gumbel,"[0.05, 0.25, 0.25]",7.0,fisherz,2.0,10.0,10.0,1.0,1000.0,linear,None,NaN,NaN,"(-1, 2)",45.0,0.0,45.0,0.0,45.0,0.0
13,0.933333,0.0,1.0,0.875000,1.000000,37.0,7.0,0.545455,0.600000,4.0,4.0,1.0,0.0,0.750000,1.000000,0.428571,0.428571,92.0,93.0,3.0,3.0,1.0,NaN,0.875000,0.0,2.0,0.7

In [26]:
data.describe()

,base_adj_f1_score,base_adj_false_negative,base_adj_false_positive,base_adj_precision,base_adj_recall,base_adj_true_negative,base_adj_true_positive,base_arrow_f1_score,base_arrow_f1_score_ce,base_arrow_false_negative,base_arrow_false_negative_ce,base_arrow_false_positive,base_arrow_false_positive_ce,base_arrow_precision,base_arrow_precision_ce,base_arrow_recall,base_arrow_recall_ce,base_arrow_true_negative,base_arrow_true_negative_ce,base_arrow_true_positive,base_arrow_true_positive_ce,base_base_shd,base_sid_lower,cluster_adj_f1_score,cluster_adj_false_negative,cluster_adj_false_positive,cluster_adj_precision,cluster_adj_recall,cluster_adj_true_negative,cluster_adj_true_positive,cluster_arrow_f1_score,cluster_arrow_f1_score_ce,cluster_arrow_false_negative,cluster_arrow_false_negative_ce,cluster_arrow_false_positive,cluster_arrow_false_positive_ce,cluster_arrow_precision,cluster_arrow_precision_ce,cluster_arrow_recall,cluster_arrow_recall_ce,cluster_arrow_true_negative,cluster_arrow_true_negative_ce,cluster_arrow_true_positive,cluster_arrow_true_positive_ce,cluster_cluster_shd,cluster_sid_lower,pruned_base_adj_f1_score,pruned_base_adj_false_negative,pruned_base_adj_false_positive,pruned_base_adj_precision,pruned_base_adj_recall,pruned_base_adj_true_negative,pruned_base_adj_true_positive,pruned_base_arrow_f1_score,pruned_base_arrow_f1_score_ce,pruned_base_arrow_false_negative,pruned_base_arrow_false_negative_ce,pruned_base_arrow_false_positive,pruned_base_arrow_false_positive_ce,pruned_base_arrow_precision,pruned_base_arrow_precision_ce,pruned_base_arrow_recall,pruned_base_arrow_recall_ce,pruned_base_arrow_true_negative,pruned_base_arrow_true_negative_ce,pruned_base_arrow_true_positive,pruned_base_arrow_true_positive_ce,pruned_base_pruned_base_shd,pruned_base_sid_lower,alpha,cluster_connectivity,empty_graph_shd,n_clusters,n_edges,n_nodes,noise_scale,sample_size,true_sid_lower,true_sid_upper,base_total_true,base_total_false,pruned_base_total_true,pruned_base_total_false,cluster_total_true,cluster_total_false
count,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,72.000000,72.000000,77.000000,77.000000,77.000000,77.000000,74.000000,74.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,0.0,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,0.0,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,72.000000,72.000000,77.000000,77.000000,77.000000,77.000000,74.000000,74.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,0.0,77.000000,77.000000,77.000000,77.000000,77.0,77.0,77.0,77.0,0.0,0.0,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,0.857686,0.974026,1.714286,0.849513,0.894259,34.792208,7.519481,0.552465,0.646544,4.597403,3.623377,2.363636,0.896104,0.698679,0.820311,0.465641,0.516709,89.142857,91.584416,3.896104,3.896104,3.584416,NaN,0.838410,0.831169,2.493506,0.808293,0.910194,34.012987,7.662338,0.761624,0.894459,1.974026,1.142857,2.454545,0.324675,0.795983,0.954885,0.782140,0.857863,89.051948,92.012987,6.519481,6.519481,3.649351,NaN,0.857686,0.974026,1.714286,0.849513,0.894259,34.792208,7.519481,0.552465,0.646544,4.597403,3.623377,2.363636,0.896104,0.698679,0.820311,0.465641,0.516709,89.142857,91.584416,3.896104,3.896104,3.584416,NaN,0.177532,0.748485,8.493506,3.194805,10.0,10.0,1.0,1000.0,NaN,NaN,42.311688,2.688312,42.311688,2.688312,41.675325,3.324675
std,0.116207,0.945544,2.383370,0.178604,0.101565,2.983894,1.759264,0.191720,0.191376,2.289908,1.899023,2.964301,1.142302,0.284213,0.236257,0.235067,0.247611,3.538191,2.267335,2.100028,2.100028,3.092226,NaN,0.134550,0.849188,3.275199,0.203949,0.089256,3.704526,1.766727,0.190241,0.160199,1.739434,1.484253,3.477542,0.895038,0.234278,0.119404,0.194908,0.192632,3.966624,2.111955,1.984217,1.984217,3.701942,NaN,0.1

In [5]:
prec_recall_metrics = ['base_adj_precision', 'pruned_base_adj_precision', 'cluster_adj_precision', \
           'base_adj_recall', 'pruned_base_adj_recall','cluster_adj_recall', \
           'base_adj_f1_score', 'pruned_base_adj_f1_score', 'cluster_adj_f1_score']
data[prec_recall_metrics].describe()

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
count,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,0.849513,0.849513,0.808293,0.894259,0.894259,0.910194,0.857686,0.857686,0.838410
std,0.178604,0.178604,0.203949,0.101565,0.101565,0.089256,0.116207,0.116207,0.134550
min,0.388889,0.388889,0.357143,0.666667,0.666667,0.700000,0.500000,0.500000,0.500000
25%,0.750000,0.750000,0.666667,0.800000,0.800000,0.857143,0.800000,0.800000,0.777778
50%,0.888889,0.888889,0.875000,0.888889,0.888889,0.900000,0.880000,0.880000,0.869565
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.941176,0.941176,0.933333
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
count_metrics = ['base_adj_false_negative', 'pruned_base_adj_false_negative', 'cluster_adj_false_negative', \
                 'base_adj_false_positive', 'pruned_base_adj_false_positive','cluster_adj_false_positive', \
                    'base_adj_true_negative', 'pruned_base_adj_true_negative', 'cluster_adj_true_negative', \
                        'base_adj_true_positive', 'pruned_base_adj_true_positive', 'cluster_adj_true_positive']
data[count_metrics]

,base_adj_false_negative,pruned_base_adj_false_negative,cluster_adj_false_negative,base_adj_false_positive,pruned_base_adj_false_positive,cluster_adj_false_positive,base_adj_true_negative,pruned_base_adj_true_negative,cluster_adj_true_negative,base_adj_true_positive,pruned_base_adj_true_positive,cluster_adj_true_positive
0,2.0,2.0,2.0,0.0,0.0,0.0,37.0,37.0,37.0,6.0,6.0,6.0
1,0.0,0.0,0.0,1.0,1.0,1.0,37.0,37.0,37.0,7.0,7.0,7.0
2,2.0,2.0,1.0,0.0,0.0,0.0,34.0,34.0,34.0,9.0,9.0,10.0
3,2.0,2.0,1.0,6.0,6.0,9.0,33.0,33.0,30.0,4.0,4.0,5.0
4,2.0,2.0,1.0,5.0,5.0,6.0,30.0,30.0,29.0,8.0,8.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...
72,1.0,1.0,1.0,0.0,0.0,1.0,36.0,36.0,35.0,8.0,8.0,8.0
73,2.0,2.0,2.0,5.0,5.0,7.0,31.0,31.0,29.0,7.0,7.0,7.0
74,1.0,1.0,1.0,7.0,7.0,10.0,29.0,29.0,26.0,8.0,8.0,8.0
75,1.0,1.0,0.0,0.0,0.0,0.0,36.0,36.0,36.0,8.0,8.0,9.0


In [7]:
dist_metrics = ['base_base_shd', 'pruned_base_pruned_base_shd', 'cluster_cluster_shd']#, \
         #   'base_sid_lower', 'base_sid_upper', 'pruned_base_sid_lower', 'pruned_base_sid_upper', \
         #      'cluster_sid_lower', 'cluster_sid_upper']
data[dist_metrics].describe()

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
count,77.000000,77.000000,77.000000
mean,3.584416,3.584416,3.649351
std,3.092226,3.092226,3.701942
min,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000
50%,3.000000,3.000000,2.000000
75%,5.000000,5.000000,5.000000
max,15.000000,15.000000,15.000000


In [8]:
edge_ratios = data['edge_ratios'].apply(pd.Series)
edge_ratios.columns = ['intra_cluster', 'inter_cluster', 'inter_including_disconnected_cluster']
mean_values = edge_ratios.mean()
mean_values = pd.DataFrame([mean_values])
print('Edge Ratios \n', mean_values)

Edge Ratios 
    intra_cluster  inter_cluster  inter_including_disconnected_cluster
0       0.224805       0.299481                              0.203247


### Comparing true & false positives and true & false negatives

In [9]:
true_false_positive_metrics = ['base_adj_true_positive', 'pruned_base_adj_true_positive', 'cluster_adj_true_positive', \
                          'base_adj_false_positive', 'pruned_base_adj_false_positive','cluster_adj_false_positive']

true_false_negative_metrics = ['base_adj_true_negative', 'pruned_base_adj_true_negative', 'cluster_adj_true_negative', \
                          'base_adj_false_negative', 'pruned_base_adj_false_negative', 'cluster_adj_false_negative']

true_false_metrics = ['base_adj_true_positive', 'pruned_base_adj_true_positive', 'cluster_adj_true_positive', \
                    'base_adj_true_negative', 'pruned_base_adj_true_negative', 'cluster_adj_true_negative', \
                    'base_adj_false_positive', 'pruned_base_adj_false_positive','cluster_adj_false_positive', \
                    'base_adj_false_negative', 'pruned_base_adj_false_negative', 'cluster_adj_false_negative']

In [10]:
data[true_false_positive_metrics].describe()

,base_adj_true_positive,pruned_base_adj_true_positive,cluster_adj_true_positive,base_adj_false_positive,pruned_base_adj_false_positive,cluster_adj_false_positive
count,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,7.519481,7.519481,7.662338,1.714286,1.714286,2.493506
std,1.759264,1.759264,1.766727,2.383370,2.383370,3.275199
min,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000
25%,7.000000,7.000000,7.000000,0.000000,0.000000,0.000000
50%,8.000000,8.000000,8.000000,1.000000,1.000000,1.000000
75%,8.000000,8.000000,9.000000,2.000000,2.000000,4.000000
max,12.000000,12.000000,12.000000,11.000000,11.000000,12.000000


In [11]:
data[true_false_negative_metrics].describe()

,base_adj_true_negative,pruned_base_adj_true_negative,cluster_adj_true_negative,base_adj_false_negative,pruned_base_adj_false_negative,cluster_adj_false_negative
count,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,34.792208,34.792208,34.012987,0.974026,0.974026,0.831169
std,2.983894,2.983894,3.704526,0.945544,0.945544,0.849188
min,27.000000,27.000000,24.000000,0.000000,0.000000,0.000000
25%,34.000000,34.000000,32.000000,0.000000,0.000000,0.000000
50%,35.000000,35.000000,35.000000,1.000000,1.000000,1.000000
75%,36.000000,36.000000,36.000000,2.000000,2.000000,1.000000
max,43.000000,43.000000,43.000000,3.000000,3.000000,3.000000


### Comparing total true and false

In [12]:
data['base_total_true'] = data['base_adj_true_positive'] + data['base_adj_true_negative']
data['base_total_false'] = data['base_adj_false_positive'] + data['base_adj_false_negative']
data['pruned_base_total_true'] = data['pruned_base_adj_true_positive'] + data['pruned_base_adj_true_negative']
data['pruned_base_total_false'] = data['pruned_base_adj_false_positive'] + data['pruned_base_adj_false_negative']
data['cluster_total_true'] = data['cluster_adj_true_positive'] + data['cluster_adj_true_negative']
data['cluster_total_false'] = data['cluster_adj_false_positive'] + data['cluster_adj_false_negative']
total_true_false_metrics = ['base_total_true', 'pruned_base_total_true', 'cluster_total_true', \
                    'base_total_false', 'pruned_base_total_false', 'cluster_total_false']
data[total_true_false_metrics].describe()

,base_total_true,pruned_base_total_true,cluster_total_true,base_total_false,pruned_base_total_false,cluster_total_false
count,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,42.311688,42.311688,41.675325,2.688312,2.688312,3.324675
std,2.461598,2.461598,3.278485,2.461598,2.461598,3.278485
min,34.000000,34.000000,32.000000,0.000000,0.000000,0.000000
25%,42.000000,42.000000,41.000000,1.000000,1.000000,1.000000
50%,43.000000,43.000000,43.000000,2.000000,2.000000,2.000000
75%,44.000000,44.000000,44.000000,3.000000,3.000000,4.000000
max,45.000000,45.000000,45.000000,11.000000,11.000000,13.000000


### Grouping by clusters

In [13]:
data.groupby('n_clusters').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
n_clusters,,,,,,,,,
2.0,0.852771,0.852771,0.819406,0.884865,0.884865,0.910859,0.855372,0.855372,0.845151
3.0,0.840468,0.840468,0.795801,0.902645,0.902645,0.915717,0.860347,0.860347,0.840415
4.0,0.808244,0.808244,0.747390,0.859723,0.859723,0.866390,0.813355,0.813355,0.772191
5.0,0.894518,0.894518,0.861126,0.938077,0.938077,0.946410,0.903626,0.903626,0.888877


In [14]:
data.groupby('n_clusters').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
n_clusters,,,
2.0,3.433333,3.433333,3.833333
3.0,4.117647,4.117647,3.823529
4.0,4.266667,4.266667,4.733333
5.0,2.600000,2.600000,2.000000


### Grouping by alpha

In [15]:
data.groupby('alpha').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
alpha,,,,,,,,,
0.01,0.978525,0.978525,0.973693,0.846376,0.846376,0.877748,0.904745,0.904745,0.920193
0.05,0.983333,0.983333,0.965476,0.915207,0.915207,0.921873,0.944413,0.944413,0.939930
0.10,0.890236,0.890236,0.843555,0.878064,0.878064,0.892458,0.875651,0.875651,0.857665
0.25,0.746551,0.746551,0.685906,0.894906,0.894906,0.913425,0.809668,0.809668,0.776248
0.50,0.631719,0.631719,0.550780,0.943126,0.943126,0.949792,0.747676,0.747676,0.687112


In [16]:
data.groupby('alpha').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
alpha,,,
0.01,2.294118,2.294118,1.470588
0.05,1.400000,1.400000,1.000000
0.10,2.333333,2.333333,2.333333
0.25,4.333333,4.333333,5.000000
0.50,7.733333,7.733333,8.733333


### Grouping by distribution type


In [17]:
data.groupby('distribution_type').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
distribution_type,,,,,,,,,
exp,0.847038,0.847038,0.803942,0.884321,0.884321,0.923822,0.851241,0.851241,0.839709
gauss,0.868094,0.868094,0.835981,0.902305,0.902305,0.906151,0.874553,0.874553,0.855837
gumbel,0.832763,0.832763,0.784022,0.896227,0.896227,0.900227,0.846845,0.846845,0.818936


In [18]:
data.groupby('distribution_type').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
distribution_type,,,
exp,3.423077,3.423077,3.653846
gauss,3.307692,3.307692,3.269231
gumbel,4.040000,4.040000,4.040000


### Grouping by DAG method

In [19]:
data.groupby('dag_method').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
dag_method,,,,,,,,,
erdos_renyi,0.843940,0.843940,0.802412,0.895780,0.895780,0.910675,0.854350,0.854350,0.834242
scale_free,0.869181,0.869181,0.829047,0.888889,0.888889,0.908497,0.869458,0.869458,0.853121


In [20]:
data.groupby('dag_method').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
dag_method,,,
erdos_renyi,3.733333,3.733333,3.666667
scale_free,3.058824,3.058824,3.588235


### Grouping by #clusters

In [21]:
data.groupby('n_clusters').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
n_clusters,,,,,,,,,
2.0,0.852771,0.852771,0.819406,0.884865,0.884865,0.910859,0.855372,0.855372,0.845151
3.0,0.840468,0.840468,0.795801,0.902645,0.902645,0.915717,0.860347,0.860347,0.840415
4.0,0.808244,0.808244,0.747390,0.859723,0.859723,0.866390,0.813355,0.813355,0.772191
5.0,0.894518,0.894518,0.861126,0.938077,0.938077,0.946410,0.903626,0.903626,0.888877


In [22]:
data.groupby('n_clusters').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
n_clusters,,,
2.0,3.433333,3.433333,3.833333
3.0,4.117647,4.117647,3.823529
4.0,4.266667,4.266667,4.733333
5.0,2.600000,2.600000,2.000000


### Grouping by #edges

In [23]:
data.groupby('n_edges').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
n_edges,,,,,,,,,
10.0,0.849513,0.849513,0.808293,0.894259,0.894259,0.910194,0.857686,0.857686,0.83841


In [24]:
data.groupby('n_edges').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
n_edges,,,
10.0,3.584416,3.584416,3.649351
